MC methods 

we dont know the p(s',r |s , a)

We merely get samples of 

    S_0 , A_0 , R_1, S_1 , A_1 , R_2 , S_2 , A_2

after running some policy through the MDP

MC use:

- average to approximate expected values

- GPI to obtain pi =~ pi_*


MC does not estimate p(s' , r|s, a) and proceed with GPI as though it's known

Model-Free vs Model-Based Method 



Model is to plan actions before they are taken 

In [1]:
## Monte CArlo Evaluation


# Gaol: Given sampels under pi , estimate q_pi
# we can express q_pi estimation as v_pi estimation
# by S-new_t = (S_t, A_t )
# 
# Any evaluation algorithm estimating v(s-new) would be estimating q_pi(s,a) 

# M trajectory samples of an MRP 

In [127]:
import numpy as np


class SimpleEnvironement():
    def __init__(self):

        self.states_type = ['no' , -2 , -1, 0, 1, 2, 'set' ]
        self.action_type = ['up', 'down']
        self.action = [1, -1]
        self.states_idx = 3
        self.episodes = [] # [(states , reward)] 


    def start_seq(self):
        return self.action

    def step(self, act):
        self.states_idx += act
        reward = 0
        terminal = False
        if self.states_idx == len(self.states_type)-1 or self.states_idx == 0:
            terminal = True
            if self.states_idx == len(self.states_type)-1:
                reward = 1
        next_state = self.states_type[self.states_idx]

        self.episodes.append((next_state, reward))
        return next_state, reward , terminal
    
    def reset(self):
        self.episode = [(0,0)]
        self.states_idx = 3 
        return 0 , self.action

In [128]:
env = SimpleEnvironement()

print(len(env.states_type))

_, action = env.reset()

7


In [37]:
action

[1, -1]

In [40]:
next_state , reward , terminal = env.step(1)

next_state, reward, terminal

('set', 1, True)

In [41]:
env.episodes

[(1, 0), (2, 0), ('set', 1)]

In [23]:
next_state, reward, terminal

(1, 0, False)

In [140]:
import numpy as np


class SimpleEnvironement():
    def __init__(self):

        self.states_type = ['no' , -2 , -1, 0, 1, 2, 'set' ]
        self.action_type = ['up', 'down']
        self.action = [1, -1]
        self.states_idx = 3
        self.episodes = [] # [(states , reward)]


    def start_seq(self):
        return self.action

    def step(self, act):
        self.states_idx += act
        reward = 0
        terminal = False
        if self.states_idx == len(self.states_type)-1 or self.states_idx == 0:
            terminal = True
            if self.states_idx == len(self.states_type)-1:
                reward = 1
        next_state = self.states_type[self.states_idx]

        self.episodes.append((next_state, reward))
        return next_state, reward , terminal
    
    def reset(self):
        # FIX: Changed 'self.episode' to 'self.episodes' to clear the correct list
        self.episodes = [] 
        self.states_idx = 3 # Resetting the starting position
        return 0 , self.action # The return values here aren't used in the main loop example I provided, but the method works.
    
    
    
class MonteCarloEvaluation():
    def __init__(self, actions):
        self.discount_rate = 1
        self.val_states = { 
            -2: 0.5 ,
            -1: 0.5 ,
            0: 0.5 ,
            1: 0.5,
            2: 0.5,
        }
        self.action_prob = [1/2 , 1/2]
        self.actions = [1, -1] 
        self.alpha = 0.05
        
    def policy(self):
        return np.random.choice(self.actions, p=self.action_prob)
    
    def update(self, episodes):
        G_t = 0
        for t in reversed(episodes):
            states_id = t[0]
            reward = t[1]

            G_t = reward + self.discount_rate * G_t
            if states_id in self.val_states.keys():
                self.val_states[states_id] += self.alpha * ( G_t -  self.val_states[states_id])
                
 

In [ ]:
import random
class epilson_greedy_MC(MonteCarloEvaluation):
    def __init__(self,actions):
        super().__init__(actions) 
        self.epilson = 0.4

        self.q_values = {}
        for state in [-2, -1, 0, 1, 2]:
            for action in [1, -1]:
                self.q_values[(state, action)] = 0.5


    def e_greedy_func(self):
        rand = random.random()
        if rand > self.epilson:
            return
        return self.random_policy()
    
    def get_best_action(self,state):
        self.candidate = {action: self.q_value[(state , action)] for action in self.actions}
        max_key = max( self.candidate, key= self.candidate.get)
        return max_key

    def update(self, episodes):
        G_t = 0
        for states_id, action, reward in reversed(episodes):

            G_t = reward + self.discount_rate * G_t
            if (states_id,action) in self.q_values.keys():
                self.q_values[(states_id , action)] += self.alpha * ( G_t -  self.q_values[(states_id , action)])
                
 



In [142]:
import numpy as np

# (Assume SimpleEnvironement and MonteCarloEvaluation classes are defined as in the previous, corrected example)

env = SimpleEnvironement()
_ , actions = env.reset()
actor = MonteCarloEvaluation(actions=actions)

# --- Start of one episode ---
# The environment always starts at state 0 value (index 3)
current_state = 0 
done = False
episode_num = 10 

for i in range(500):
    env.reset() 
    done = False

    while not done:
        # 1. Store the state before taking an action
        # We store the *value* of the state for simplicity with the dict lookup
        state_value_at_step = env.states_type[env.states_idx]

        # 2. Agent decides an action
        action = actor.policy()
        # 3. Environment steps forward
        next_state_value, reward, done = env.step(action)
      
    # 6. Use the collected history to update the actor's value estimates
    actor.update(env.episodes)

    # Print updated values to check learning
    print("Updated State Values:", actor.val_states)
    env.reset()
    

Updated State Values: {-2: 0.5, -1: 0.5, 0: 0.5, 1: 0.5487500000000001, 2: 0.5487500000000001}
Updated State Values: {-2: 0.5, -1: 0.5, 0: 0.5, 1: 0.5927468750000001, 2: 0.5927468750000001}
Updated State Values: {-2: 0.475, -1: 0.475, 0: 0.5, 1: 0.5927468750000001, 2: 0.5927468750000001}
Updated State Values: {-2: 0.4286875, -1: 0.4286875, 0: 0.475, 1: 0.5349540546875, 2: 0.5631095312500001}
Updated State Values: {-2: 0.407253125, -1: 0.407253125, 0: 0.45125, 1: 0.508206351953125, 2: 0.5631095312500001}
Updated State Values: {-2: 0.38689046875, -1: 0.38689046875, 0: 0.45125, 1: 0.508206351953125, 2: 0.5631095312500001}
Updated State Values: {-2: 0.446668648046875, -1: 0.5006184548623047, 0: 0.5530396953125, 1: 0.59943099995552, 2: 0.605706351953125}
Updated State Values: {-2: 0.42433521564453125, -1: 0.45180815551323, 0: 0.4991183250195312, 1: 0.569459449957744, 2: 0.605706351953125}
Updated State Values: {-2: 0.36381440551322997, -1: 0.3680005664664978, 0: 0.4504542883301269, 1: 0.488